# SageMaker JumpStart

## Setup

Upgrade some libraries for running this notebook.

In [ ]:
!pip install sagemaker --upgrade
!pip install ipywidgets

In order to train and host with Amazon Sagemaker, you need to set up and authenticate to use AWS services.

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

print('-----AWS authentication info------')
print('Role: ' + aws_role)
print('Region: ' + aws_region)
print('----------------------------------')

## Select a pre-trained model

You can either proceed with the default model or select a different model from the dropdown that is generated when running the next cell.
You can also view a list of all JumpStart models from [JumpStart Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html).

Select the following 2 information from a model_manifest file of JumpStart.

* Model ID
* Model Version

In [ ]:
import IPython
from ipywidgets import Dropdown

# Download a model_manifest file of JumpStart.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

# Select an image classification model from the manifest file.
ic_models_all_versions, ic_models = [
    model["model_id"] for model in model_list if "-ic-" in model["model_id"]
], []
[ic_models.append(model) for model in ic_models_all_versions if model not in ic_models]

# Show a dropdown list to select the models easily.
dropdown = Dropdown(
    options=ic_models,
    value=model_id,
    description="JumpStart Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(IPython.display.Markdown("## Select a JumpStart pre-trained model from the dropdown below"))
display(dropdown)

## Transfer Learninig the pre-trained model usinig your own datasets

Let's fine-tune a model for custom data sets with any number of classes.

Retrieve the following 3 information using SageMaker URI for training.
* Docker image URI
* Script URI
* Model URI


In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.utils import name_from_base

# model_version="*" shows a latest version of the model.
model_id, model_version = dropdown.value, "*"

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

training_instance_type = "ml.g4dn.2xlarge"

# Retrieve a docker image uri for a transfer learning.
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="training",
    model_id=model_id,
    model_version=model_version,
    instance_type=training_instance_type,
)

# Retrieve scripts and other data for a transfer learning.
train_source_uri = script_uris.retrieve(
    model_id=model_id, 
    model_version=model_version, 
    script_scope="training"
)

# Retrieve a base model uri for a transfer learning.
base_model_uri = model_uris.retrieve(
    model_id=model_id, 
    model_version=model_version, 
    model_scope="training"
)

print('----------JumpStart info----------')
print('Endpoint name: ' + endpoint_name)
print('Container image uri: ' + train_image_uri)
print('Source uri: ' + train_source_uri)
print('Base model uri: ' + base_model_uri)
print('----------------------------------')

Use rose or dandelion image classification task datasets as custom dataset for transfer learning like below.

    Input_directory
    ├ roses
    │   ├ r-0001.jpg
    │   ・・・
    │   └ r-xxxx.jpg
    └ dandelion
        ├ d-0001.jpg
        ・・・
        └ d-yyyy.jpg
    
You can download the dataset from [here](https://www.tensorflow.org/datasets/catalog/tf_flowers).

In [ ]:
# S3 input path
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_prefix = "training-datasets/tf_flowers/"
training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

# S3 output path
output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-ic-training"
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

In [ ]:
# Retrieve default model parameters for transfer learning.
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)
print(hyperparameters)

## Check a script of the transfer learning

In [ ]:
# download sourcedir from S3
import os
import shutil

if not os.path.exists('source'):
    os.makedirs('source')
sourcedir_path = train_source_uri.replace('s3://' + training_data_bucket + '/','')
boto3.client("s3").download_file(training_data_bucket, sourcedir_path, './source/sourcedir.tar.gz')
shutil.unpack_archive('./source/sourcedir.tar.gz', extract_dir='./source/')

In [ ]:
!pygmentize ./source/transfer_learning.py

## Run a Training Job
Create an Estimator object containing all required arguments and run a training job.

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

# Define an Estimator class
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=base_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

# Run the Training Job 
ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

## Deploy Endpoints

Retrieve the following 2 information using SageMaker URI for inference.
* Docker image URI
* Script URI

Generally, you must retrieve a Model URI as well. In this case, you will use the transfer-learned model instead of the Model URI.

In [ ]:
inference_instance_type = "ml.m5.xlarge"
endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")

# Rerieve a docker image for an inference
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Rerieve a script and other data for an inference
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

Deploy an endpoint using the transfer-learned model.

In [ ]:
finetuned_predictor = ic_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

Download sample images of roses and sunflowers from the S3 bucket.

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
key_prefix = "training-datasets/tf_flowers"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


flower_images = {
    "img1.jpg": "roses/10503217854_e66a804309.jpg",
    "img2.jpg": "sunflowers/1008566138_6927679c8a.jpg",
}
download_from_s3(flower_images)

Predict labels from the downloaded iamges via the endpoint.

In [ ]:
from IPython.core.display import HTML

for image_filename in flower_images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
        )
    )

Delete the model and the endpont

In [ ]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

## Incremental learinig

Incremental learning is a learning approach that trains a new model using an extended dataset that includes patterns that were not considered in the previous learning, leading to a degradation in the model's performance.

You can start to train a new model using artifacts from an existing model and an extended dataset. Incremental learning can save both time and resources by avoiding the need to retrain the model from scratch.

In [ ]:
# Based on the job name for training and the output location of the artifacts,
# identify the trained model in the previous step.

last_training_job_name = ic_estimator._current_job_name

last_trained_model_path = f"{s3_output_location}/{last_training_job_name}/output/model.tar.gz"

print('-------Previous step info---------')
print('Last training job name: ' + last_training_job_name)
print('Last trained model path: ' + last_trained_model_path)
print('----------------------------------')

In [ ]:
incremental_train_output_prefix = "jumpstart-example-ic-incremental-training"
incremental_s3_output_location = f"s3://{output_bucket}/{incremental_train_output_prefix}/output"
incremental_training_job_name = name_from_base(f"jumpstart-example-{model_id}-incremental-training")

print('----------New step info-----------')
print('Incremental training job name: ' + incremental_training_job_name)
print('Incremental training output path: ' + incremental_s3_output_location)
print('----------------------------------')

incremental_train_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=last_trained_model_path,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=incremental_s3_output_location,
    base_job_name=incremental_training_job_name,
)

incremental_train_estimator.fit({"training": training_dataset_s3_path}, logs=True)

After completing training, you can deploy the incremental-learned model following the previous deploy steps